In [1]:
import numpy as np
from bokeh.plotting import *
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource, CustomJS, Slider, Button, DataTable, TableColumn, NumberFormatter
from bokeh.io import curdoc
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as pat
from math import pi
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
output_notebook()
output_file("index.html", title='Two Flavor Neutrino Oscillations')
%matplotlib inline

Loading BokehJS ...

In [2]:
# prepare some data
N = 300
x = np.linspace(0, 4*np.pi, N)
k = np.cos(x)
f = np.linspace(-1, 1, N)
g = (np.sin(x))**2
w = -1
z = 0.1
m = np.linspace(-1, 1, 300)
n = []

for i in range(300):
    n.append(0.5)

a = -1
b = 0.1

u = -1
v = 0.5

# create a column data source for the plots to share
d = {};
d['x'] = x

d1 = {};
d1['w'] = [w]
d1['z'] = [z]

d4 = {};
d4['a'] = [a]
d4['b'] = [b]

d6 = {};
d6['u'] = [u]
d6['v'] = [v]


for i in range(11):
    xi = np.cos(x)
    yi = np.sin(x)
    d['x'+str(i)] = xi
    d['y'+str(i)] = yi
    
circ1Colors = ['blue' for x in f]

# define sources


source = ColumnDataSource(data=d)
source1 = ColumnDataSource(data=d1)
source2 = ColumnDataSource(data=dict(f=f, g=g, color=circ1Colors))
source3 = ColumnDataSource(data=dict(m=m, n=n))
source4 = ColumnDataSource(data=d4)
source5 = ColumnDataSource(data=dict(
    start=[2*pi, 0],end=[0, 2*pi], color=['purple', 'green'],legend=['muon neutrino','electron neutrino']
))
source6 = ColumnDataSource(data=d6)
                            
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300, title=None,x_axis_label='Length',y_axis_label='Length')
right = figure(tools=TOOLS, width=300, height=300, title=None,x_axis_label='Length',y_axis_label="ν_e Survival Probability")
bot = figure(tools=TOOLS, width=300, height=300, title='Density Function',x_axis_label='Length',y_axis_label='A(x)')
pie = figure(height=300,width=300,title='Neutrino Flavor Transition')


# plotting
cir1 = right.circle('f', 'g', source=source2, fill_color='color', line_color='color')
cir = left.circle('w', 'z', source=source1, color='blue', line_width=1, line_alpha=0.4,size=15)
cir4 = right.circle('a', 'b', source=source4, color='blue', line_width=1, line_alpha=0.4,size=15)
cir5 = bot.circle('u', 'v', source=source6, color='blue', line_width=1, line_alpha=0.4,size=15)

bot.line('m', 'n', source=source3, color='blue', line_width=2, line_alpha=0.1)
bot.circle('m', 'n', source=source3, color='blue',size=2)
pie.wedge(x=0, y=0, start_angle='start', end_angle='end', radius=.4,
        color='color', alpha=0.6, source=source5,legend='legend')


#changing circles color
for i in range(11):
    if i%2==0:
        left.line('x'+str(i), 'y'+str(i), source=source, color='red', line_width=4, line_alpha=1)
        #left.circle('x'+str(i), 'y'+str(i), source=source, color='red')
    else:
        left.line('x'+str(i), 'y'+str(i), source=source, color='blue', line_width=4, line_alpha=0.4)
        #left.circle('x'+str(i), 'y'+str(i), source=source, color='blue')
        
        


callback = CustomJS(args=dict(cir=cir,cir1=cir1,cir4=cir4,cir5=cir5, 
                              source=source, source1=source1,source2=source2,source3=source3,
                              source4=source4, source5=source5,source6=source6), code="""
                             
// DEFINES DATA

    var data = source.data;
    var data1 = source1.data;
    var data2 = source2.data;
    var data3 = source3.data;
    var data4 = source4.data;
    var data5 = source5.data;
    var data6 = source6.data;


// DEFINES VARIABLES

    var M = circle.value;
    var x = data['x']
    var w = data1['w']
    var z = data1['z']
    var a = data4['a']
    var b = data4['b']
    var s = slope.value
    var t = time.value
    var f = data2['f']
    var g = data2['g']
    var m = data3['m']
    var n = data3['n']
    var u = data6['u']
    var v = data6['v']


//1. DENSITY FUNCTION

//heaviside function for javascript    
    function side(x1, x2) {
        if(x1<0){
            return 0;
        }
        else if(x1==0){
            return x2;
        }
        else{
            return 1;
        }
      }
      
//setup for density function    
    for (j = 0; j < m.length; j++){
                    n[j] = 0.5
                }
//makes and changes density function    
    for (i =1; i< M;i++){
            if (i%2==1){
                for (j = 0; j < m.length; j++){
                    n[j] = n[j] + side(m[j]+(1-i/M),1)
                    n[j] = n[j] -side(m[j]-(1-i/M),1)
                }
            }

            else{
                for (j = 0; j < m.length; j++){
                    n[j] = n[j] + side(m[j]-(1-i/M),1)
                    n[j] = n[j] - side(m[j]+(1-i/M),1)
                }
            }

        }

//moves particle on density curve
    u[0] = (2*t/300)-1;
    v[0] = n[t];

//changes color of particle on density curve  
    if ((Math.sin(.5*M*((2*t/300)-1)))*(Math.sin(.5*M*((2*t/300)-1))) > 0.5){
        cir5.glyph.fill_color = "green";
    }
    else{
        cir5.glyph.fill_color = "purple";
    }


//2. PIE CHART

//changes pie chart     
    data5['end'][1] = 2*3.14159*(Math.sin(.5*M*((2*t/300)-1)))*(Math.sin(.5*M*((2*t/300)-1)))
    
    


//3. PROBABILITY CURVE

//changes color and shape of proba curve     
    for (i = 0; i < f.length; i++) {
        g[i] = (Math.sin(.5*M*f[i]))*(Math.sin(.5*M*f[i]));
        if (g[i] > 0.5){
            data2.color[i] = "green";
        }
        else{
            data2.color[i] = "purple";
        }
    }

    


//moves particle on proba curve
    a[0] = (2*t/300)-1;
    b[0] = (Math.sin(.5*M*a[0]))*(Math.sin(.5*M*a[0]));
    
//changes color of particle on proba curve  
    if ((Math.sin(.5*M*((2*t/300)-1)))*(Math.sin(.5*M*((2*t/300)-1))) > 0.5){
        cir4.glyph.fill_color = "green";
    }
    else{
        cir4.glyph.fill_color = "purple";
    }
    



//4. CIRCLES GRAPH

//changes particle color on circle graph    
    if (g[t] > 0.5){
        cir.glyph.fill_color = "green";
    }
    else{
        cir.glyph.fill_color = "purple";
    }

//moves particle on circle graph
    w[0] = 2.*t/300. - 1.;
    z[0] = -s*w[0];
    
//defines data
    for (i=M; i<11; i++){
        xi = data['x'+i];
        yi = data['y'+i];
        
//gets rid off previous circles

        for (j = 0; j < x.length; j++) {
            yi[j] = 0;
            xi[j] = 0;
        }
    }

//makes circles
    for (i = 0; i < M; i++) {
        xi = data['x'+i];
        yi = data['y'+i];
        
        for (j = 0; j < x.length; j++) {
            yi[j] = (1-i/M)*Math.sin(x[j]);
            xi[j] = (1-i/M)*Math.cos(x[j]);
        }
    }
    
    

//5. SOURCE CHANGES

    source.change.emit();
    source1.change.emit();
    source2.change.emit();
    source3.change.emit();
    source4.change.emit();
    source5.change.emit();
    source6.change.emit();
""")

#make a button, with a different callback
#inside callback, javascript setInterval


#plotting setup and sliders
p = gridplot([[left, right],[bot,pie]],toolbar_location='right')

circle_slider = Slider(start=1, end=10, value=1, step=1,
                    title="number of circles", callback=callback)
callback.args["circle"] = circle_slider

slope_slider = Slider(start=.1, end=1, value=.1, step=.1,
                    title="slope", callback=callback)
callback.args["slope"] = slope_slider

time_slider = Slider(start=1, end=299, value=1, step=5,
                   title="time", callback=callback)
callback.args["time"] = time_slider

layout = row(p,
    widgetbox(circle_slider,time_slider),
)
show(layout)